In [ ]:
from databases import Database
from sketch.api import data


database = Database(
    "sqlite+aiosqlite:///../../test.db",
)

pf = await data.get_portfolio(database, 'justin')

In [ ]:
import pandas as pd
import json

test = pd.Series({k: json.dumps(v.to_dict()) for k, v in pf.sketchpads.items()})
test.to_parquet("349082_wikitable.parquet")

In [ ]:
df = test.to_frame()

In [ ]:
df.columns = ["sketchpad_data"]
df.index.name = "sketchpad_id"

In [ ]:
df.to_parquet("349082_wikitable.parquet", compression="gzip")

In [ ]:
!ls -lh 349082_wikitable.parquet

In [ ]:
# 300 MB -> 200 MB gzip compression on 349082 sketchpad scale


In [ ]:
import numpy as np
import altair as alt
cards = np.array(
    [
        x.get_sketchdata_by_name("HyperLogLog").count()
        for x in pf.sketchpads.values()
    ]
)
# hist, bins = np.histogram(cards)
upper = pow(2, np.ceil(np.log(np.max(cards) / np.log(2))))
bins = np.geomspace(1, upper, num=100)
hist, *_ = np.histogram(cards, bins=bins)
# should be geometric mean...
df = pd.DataFrame({"x": bins[:-1], "x1": bins[1:], "y": hist})
chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X(
            "x", title="Unique Count", scale=alt.Scale(type="log", domainMin=1)
        ),
        # x2="x2",
        # y=alt.Y("y", title="Number", scale=alt.Scale(type="log")),
        y=alt.Y("y", title="Count"),
    )
    .properties(width="container", height=300)
)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))

x, y = zip(*[(x, y) for x, y in zip(df['x'], df['y']) if y > 0])
x, y = np.array(x), np.array(y)

y = y/np.array([1, *(x[1:] - x[:-1])])

plt.errorbar(x, y, xerr=0.02*x, yerr=np.sqrt(y), linestyle='None', marker='.')

# print(x, y)
z = np.polyfit(np.log(x[7:]), np.log(y[7:]), 1)
fit_data = np.poly1d(z)(np.log(x))
plt.loglog(x, np.exp(fit_data), 'r-',)
# plt.plot(x, np.exp(fit_data), 'r-',)
print(z)
plt.xlim([0, max(x)])
plt.ylim([0, max(y)])
plt.plot()

In [ ]:
df = pd.DataFrame(zip(*[x, y, np.exp(fit_data)]), columns=['cardinality', 'count', 'fit'])
cardcount = lambda x: float(df.iloc[(df['cardinality'] - x).abs().argsort()[0],:]['count'])
cardcount_est = lambda x: float(df.iloc[(df['cardinality'] - x).abs().argsort()[0],:]['fit'])

In [ ]:
for i in range(0, 10):
    print(i, 2**i, cardcount_est(2**(i))/cardcount_est(2**(i+1)), cardcount_est(2**i))

In [ ]:
3.621120346441395*(6.12**)

In [ ]:
 (6.12**40)

In [ ]:
# this was cool, co-pilot played along great here. 
# print(1, 2, 41616/32095)
# print(2, 4, 32095/23511)
# print(4, 8, 23511/11155)
# print(8, 16, 11155/4455)
# print(16, 32, 4455/1398) # co-pilot just guessed 4455/1331, but right answer is 1398 lol
# print(32, 64, 1398/590) # co pilot typed "32, 64, 1398/397" but right answer is 590 (??)
# print(64, 128, 590/273) # co pilot typed "64, 128, 590/273" but right answer is 273 (??)
# # i might have a flawed binning x-axis strategy... Need to think about that some more. 


In [ ]:
# Database consideration -> always have a parquet "rolling source" or something, something that is the serialized tape
#  -> its off of that (read many) that a bunch of things can be calculated.
#  -> can have lots of "named" rolling-sources. 

In [ ]:
2**40